<div class="alert alert-block alert-info">
<span style="font-size: 26px;"><b>Blob analytical scaling law based on tracking (Script 6_0)</b></span><br>
<span style="font-size: 16px;">
- In this script, we apply the analytical scaling laws derived in the past litteratures. <br>
- There are two existing models for this:<br>
- (1) One makes use of Lambda and Theta<br>
- (2) The other makes use of hat-a and hat-v. 
</span>
</div>



In [1]:
import sys
sys.path.append('/marconi/home/userexternal/klim0000/my_venv/lib/python3.9/site-packages')
#from skimage.measure import label, regionprops

In [2]:
sys.path.append('/marconi/home/userexternal/klim0000/miniconda3/lib/python3.9/site-packages/gbs_python/gbspy/blobs/Devel')
sys.path.append('/marconi/home/userexternal/klim0000/my_venv/lib/python3.9/site-packages')
sys.path.append('/marconi/home/userexternal/klim0000/miniconda3/lib/python3.9/site-packages/gbs_python/gbspy/blobs')
from Blob_detection import *
from CBlob_plots import *
from Filament_utils import *
from CBlobs import *

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
import gbspy as g
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py

import multiprocessing as mp
from matplotlib.animation import FuncAnimation
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from matplotlib.ticker import AutoMinorLocator
import matplotlib.lines as mlines
from matplotlib.ticker import AutoMinorLocator
import matplotlib.colors as mcolors

from skimage import measure

In [4]:
from gbspy.blobs import *

In [5]:
#import sys  
#sys.path.insert(0, '/marconi/home/userexternal/klim0000/miniconda3/lib/python3.9/site-packages/gbs_python/gbspy')

# Get the current working directory|
cwd = os.getcwd()

# PT
os.chdir('/marconi_scratch/userexternal/klim0000/GBS/DN_tria/delta_scan/DPT_d0p6')

#NT
#os.chdir('/marconi_scratch/userexternal/klim0000/GBS/DN_tria/delta_scan/DNT_dn0p6')

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Load GBS simulations
s=g.Sim()

Current working directory: /marconi/home/userexternal/klim0000/jupyter_notebook/3_GBS_DN_Tria


In [6]:
t2= s.time[-1]
t1= t2 - 0.1

# Entire torus
z1 = s.z[0]
z2 = s.z[-1]
iz = s.find_ind(z2, s.z)
bsign = s.attribute["B0sign"]


dens         = s.get_field('n', z1=z1, z2=z2, t1=t1, t2=t2)
dens_avg     = np.mean(dens, axis=(2,3)); 
dens_avg_tor = np.mean(dens, axis=2)
dens_std     = np.std(dens, axis=(2,3))

Te  = s.get_field('Te', z1=z1, z2=z2, t1=t1, t2=t2)
Te_avg  = np.mean(Te, axis=(2,3)); 
Te_avg_tor = np.mean(Te, axis=2)

pe = dens * Te
pe_avg = np.mean(pe, axis=(2,3))
pe_std = np.std(pe, axis=(2,3))

Ti  = s.get_field('Ti', z1=z1, z2=z2, t1=t1, t2=t2)
Ti_avg  = np.mean(Ti, axis=(2,3)); 
Ti_avg_tor = np.mean(Ti, axis=2)

phi = s.get_field('strmf', z1=z1, z2=z2, t1=t1, t2=t2)
phi_x = np.gradient(phi, axis=1)
phi_y = np.gradient(phi, axis=0)


# Normalized
dens_fluct   = np.sqrt((dens[:,:,0,:]-dens_avg_tor)**2) / dens_avg_tor
#dens_fluct = dens[:,:,0,-1] - dens_avg
dens_fluct_avg = np.mean(dens_fluct, axis=2)

<div class="alert alert-block alert-info">
<span style="font-size: 26px;"><b>Blob detection </b></span><br>
</div>

In [30]:
print(len(np.sum(mask, axis=(0, 1))))
print(np.sum(mask, axis=(0, 1)))

np.shape(data)

80
[79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056.
 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056.
 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056.
 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056.
 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056.
 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056.
 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056.
 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056. 79056.]


(324, 244, 80, 1)

In [37]:
### Input

mask = None


######################

if mask is None:
    mask = np.ones([s.ny, s.nx, s.nz])

if any([t is None for t in (t1, t2)]):
    its = 0
    ite = len(s.time)-1
else:
    its = s.find_ind(s.time, t1) 
    ite = s.find_ind(s.time, t2) 
    
nsteps = int(ite-its+1)



[X, Y] = np.meshgrid(s.x, s.y)

FilamentList = []
splitcount = 0; mergcount = 0 

mask_z = np.where(np.sum(mask, axis=(0, 1)) > 0)[0]
# np.sum(mask, axis=(0, 1) = sum of all the elements in nx*ny
mask1 = np.array(mask[:, :, mask_z], copy=True)
z_new = s.z[mask_z]


data = s.get_field('n', t1 = s.time[its], t2 = s.time[its])[:, :, mask_z] #4D
data2 = np.mean(data, axis=2)
data_avg = np.mean(data, axis=(2,3))
data_std = np.std(data, axis=(2,3))

fluct = get_fluct_from_data(data, data_avg, data_std)


'''
nflc = get_fluct_from_data(n_data, n_avg, n_std)
if hole == False:
    mask1[nflc[:, :, :, 0] < threshold] = 0
else:
    mask1[nflc[:, :, :, 0] > threshold] = 0

[NumObjects, PixelIdxList, TorIdxList] = get_connected_regions_torus(mask1, min_npx = len_tol)

#Return from index in z compressed (less planes) to full (all planes in gbssim)
if len(z_new) < len(gbssim.z):
    for ifil in range(NumObjects):
        for ipix in range(len(PixelIdxList[ifil])):
            PixelIdxList[ifil][ipix] = (PixelIdxList[ifil][ipix][0],
                                        PixelIdxList[ifil][ipix][1],
                                        np.argmin(np.abs(gbssim.z
                                                         - z_new[PixelIdxList[ifil][ipix][2]])))
FilamentIndeces_OLD = np.zeros(NumObjects, dtype = int)
for ib in range(NumObjects):
    FilamentList.append(Filament(gbssim, PixelIdxList[ib],
                                 TorIdxList[ib], its))
    FilamentIndeces_OLD[ib] = int(len(FilamentList) - 1)
NumObjects_OLD = NumObjects
PixelIdxList_OLD = PixelIdxList
print(f"Detection in step 1/{nsteps} done")
for it in range(its+1, ite+1):
    mask1 = np.array(mask[:, :, mask_z], copy=True)
    n_data = gbssim.get_field(which_field,
                              t1 = gbssim.time[it],
                              t2 = gbssim.time[it])[:, :, mask_z]
    nflc = get_fluct_from_data(n_data, n_avg, n_std)
    if hole == False:
        mask1[nflc[:, :, :, 0] < threshold] = 0
    else:
        mask1[nflc[:, :, :, 0] > threshold] = 0
    [NumObjects, PixelIdxList, TorIdxList] = get_connected_regions_torus(mask1,
                                                                         min_npx = len_tol)
    if len(z_new) < len(gbssim.z):
        for ifil in range(NumObjects):
            for ipix in range(len(PixelIdxList[ifil])):
                PixelIdxList[ifil][ipix] = (PixelIdxList[ifil][ipix][0],
                                            PixelIdxList[ifil][ipix][1],
                                            np.argmin(np.abs(gbssim.z
                                                             - z_new[PixelIdxList[ifil][ipix][2]])))
    FilamentIndeces = np.zeros(NumObjects, dtype = int)
    CheckOverlap = np.zeros((NumObjects_OLD, NumObjects), dtype = int)
    for ib_new in range(NumObjects):
        for ib_old in range(NumObjects_OLD):
            # To constrain the ordering of the triplets in PixelIdxList they are compared as strings
            Overlap = np.sum(np.isin([str(x) for x in PixelIdxList_OLD[ib_old]],
                                     [str(x) for x in PixelIdxList[ib_new]],
                                     assume_unique = True))
            if (Overlap > inter_tol * len(PixelIdxList_OLD[ib_old])):
                CheckOverlap[ib_old, ib_new] = 1

    [_, ib_new_over] = np.nonzero(CheckOverlap)

    alreadyadded = np.zeros(NumObjects, dtype = int)
    for iboverlap in ib_new_over:
        Sum_old = int(np.sum(CheckOverlap[:, iboverlap]))
        if (Sum_old == 1):
            ib_old = np.flatnonzero(CheckOverlap[:, iboverlap])
            ib_new = np.flatnonzero(CheckOverlap[ib_old, :])
            if (len(ib_new) == 1):
                if (not alreadyadded[ib_new]):
                    FilamentIndeces[ib_new[0]] = FilamentIndeces_OLD[ib_old[0]]
                    FilamentList[FilamentIndeces_OLD[ib_old[0]]].update(PixelIdxList[ib_new[0]],
                                                                        TorIdxList[ib_new[0]], it)
                    alreadyadded[ib_new] = 1
            else: #splitting
                for i in ib_new:
                    if (not alreadyadded[i]):
                        FilamentList.append(Filament(gbssim, PixelIdxList[i],
                                                     TorIdxList[i], it))
                        FilamentIndeces[i] = int(len(FilamentList) - 1)
                        alreadyadded[i] = 1
                        splitcount = splitcount +1

        else: #merging
            if (not alreadyadded[iboverlap]):
                FilamentList.append(Filament(gbssim, PixelIdxList[iboverlap],
                                             TorIdxList[iboverlap], it))
                FilamentIndeces[iboverlap] = int(len(FilamentList) - 1)
                alreadyadded[iboverlap] = 1
                mergcount = mergcount + 1
    ind_blob_new = np.arange(NumObjects)
    ib_new = np.setdiff1d(ind_blob_new, ib_new_over)
    for ibnon_overlap in ind_blob_new: # new blobs
        if (not alreadyadded[ibnon_overlap]):
            FilamentList.append(Filament(gbssim,
                                         PixelIdxList[ibnon_overlap],
                                         TorIdxList[ibnon_overlap], it))
            FilamentIndeces[ibnon_overlap] = int(len(FilamentList) - 1)
            alreadyadded[ibnon_overlap] = 1
    NumObjects_OLD = NumObjects
    PixelIdxList_OLD = PixelIdxList
    FilamentIndeces_OLD = FilamentIndeces
    print(f"Detection in step {it-its+1}/{nsteps} done")
nfil_tot = len(FilamentList)
FilamentList = [fil for fil in FilamentList if len(fil.itime) > age_tol]
nfil_longtime = len(FilamentList)
print('{} merging events detected'.format(mergcount))
print('{} splitting events detected'.format(splitcount))
print('Detected {} filaments in total'.format(nfil_tot))
print('{} survived more than {} steps'.format(nfil_longtime, age_tol))
return FilamentList
'''

/scratch_local/slurm_job.13028361/ipykernel_32974/2165753308.py:13: RuntimeWarning: invalid value encountered in divide
  return (data - avg4D) / std4D


'\nnflc = get_fluct_from_data(n_data, n_avg, n_std)\nif hole == False:\n    mask1[nflc[:, :, :, 0] < threshold] = 0\nelse:\n    mask1[nflc[:, :, :, 0] > threshold] = 0\n\n[NumObjects, PixelIdxList, TorIdxList] = get_connected_regions_torus(mask1, min_npx = len_tol)\n\n#Return from index in z compressed (less planes) to full (all planes in gbssim)\nif len(z_new) < len(gbssim.z):\n    for ifil in range(NumObjects):\n        for ipix in range(len(PixelIdxList[ifil])):\n            PixelIdxList[ifil][ipix] = (PixelIdxList[ifil][ipix][0],\n                                        PixelIdxList[ifil][ipix][1],\n                                        np.argmin(np.abs(gbssim.z\n                                                         - z_new[PixelIdxList[ifil][ipix][2]])))\nFilamentIndeces_OLD = np.zeros(NumObjects, dtype = int)\nfor ib in range(NumObjects):\n    FilamentList.append(Filament(gbssim, PixelIdxList[ib],\n                                 TorIdxList[ib], its))\n    FilamentIndeces_O

In [32]:
def get_fluct_from_data(data, avg, std):
    """ 
    Return fluctuation values from data as (data-avg)/std
    where avg and std have lower or equal dimension than data
    """
    if len(data.shape) == 4:
        avg4D = np.repeat(np.repeat(avg[:, :, np.newaxis], 
                                    data.shape[2], axis = 2)[:, :, :, np.newaxis], 
                          data.shape[3], axis = 3)
        std4D = np.repeat(np.repeat(std[:, :, np.newaxis], 
                                    data.shape[2], axis = 2)[:, :, :, np.newaxis], 
                          data.shape[3], axis = 3)
        return (data - avg4D) / std4D
    elif len(data.shape) == 3:
        avg3D = np.repeat(avg[:, :, np.newaxis], 
                                    data.shape[2], axis = 2)
        std3D = np.repeat(std[:, :, np.newaxis], 
                                    data.shape[2], axis = 2)
        return (data - avg3D) / std3D
    elif len(data.shape) == 2:
        return (data - avg) / std 
    else:
        return
